In [1]:
%matplotlib qt

import time
import numpy as np
from skimage.filters import gaussian_filter

from tifffile import imread, imsave
import cPickle as pickle

from spimagine import volshow
from spimagine import EllipsoidMesh, Mesh

%reload_ext autoreload
%autoreload 2

from netsurface3d import NetSurf3d

<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>
could not open /Users/jug/.spimagine
<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


/Users/jug/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pyopencl/__init__.py:207: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


# Load dataset and show in Spimagine 

In [275]:
filenames = ['/Users/jug/Repositories/GIT/BobSeg/MartinMiccai17/nuclei_with_border/groundtruth_label_gradient.tif',
             '/Users/jug/Repositories/GIT/BobSeg/MartinMiccai17/nuclei_with_border/isotropic_gradient.tif',
             '/Users/jug/Repositories/GIT/BobSeg/MartinMiccai17/nuclei_with_border/blurred_gradient.tif',
             '/Users/jug/Repositories/GIT/BobSeg/MartinMiccai17/nuclei_with_border/groundtruth_label.tif']
        
images = [None]*len(filenames)
for i in range(len(filenames)):
    images[i] = imread(filenames[i])
    print 'Image dimensions: ', images[i].shape

Image dimensions:  (512, 512, 512)
Image dimensions:  (512, 512, 512)
Image dimensions:  (512, 512, 512)
Image dimensions:  (512, 512, 512)


In [259]:
frame = 3

In [260]:
volwin = volshow(images[frame], stackUnits = [1.,1.,5.], raise_window=True)
print 'Dataset now visible in separate Spimagine window...'

Dataset now visible in separate Spimagine window...


# Load vectors sampling unit sphere 

In [289]:
# load pickeled unit sphere sampling
with open('sphere_sampling.pkl','r') as f:
    dictSphereData = pickle.load(f)
    
# sampling parameters
vectors = dictSphereData['points']
neighbors = dictSphereData['neighbors']
neighbors_of = dictSphereData['neighbors_of']
triangles = dictSphereData['triangles']

# Do segmentations using NetSurf3d

**some random cells**

In [290]:
frame = 0
K = 40
max_delta_k = 4

cell1_center = (343,297,264)
cell2_center = (337,282,277)
cell3_center = (332,263,296)
cell4_center = (340,252,273)
cell5_center = (339,236,287)


cell1 = NetSurf3d( vectors, triangles, neighbors_of, K=K, max_delta_k=max_delta_k )
cell2 = NetSurf3d( vectors, triangles, neighbors_of, K=K, max_delta_k=max_delta_k )
cell3 = NetSurf3d( vectors, triangles, neighbors_of, K=K, max_delta_k=max_delta_k )
cell4 = NetSurf3d( vectors, triangles, neighbors_of, K=K, max_delta_k=max_delta_k )
cell5 = NetSurf3d( vectors, triangles, neighbors_of, K=K, max_delta_k=max_delta_k )

In [291]:
max_rs = (images[0].shape[2] / 15,
          images[0].shape[1] / 15,
          images[0].shape[0] / 15)

min_rs = (2,2,2)

optimum = cell1.apply_to( images[frame], cell1_center, max_rs, min_radii=min_rs )
print 'Optimum energy cell 1: ', optimum
ins, outs = cell1.get_counts()
print 'Nodes in/out: ', ins, outs

optimum = cell2.apply_to( images[frame], cell2_center, max_rs ) #, min_radii=min_rs )
print 'Optimum energy cell 2: ', optimum
ins, outs = cell2.get_counts()
print 'Nodes in/out: ', ins, outs

optimum = cell3.apply_to( images[frame], cell3_center, max_rs ) #, min_radii=min_rs )
print 'Optimum energy cell 3: ', optimum
ins, outs = cell3.get_counts()
print 'Nodes in/out: ', ins, outs

optimum = cell4.apply_to( images[frame], cell4_center, max_rs ) #, min_radii=min_rs )
print 'Optimum energy cell 4: ', optimum
ins, outs = cell4.get_counts()
print 'Nodes in/out: ', ins, outs

optimum = cell5.apply_to( images[frame], cell5_center, max_rs ) #, min_radii=min_rs )
print 'Optimum energy cell 5: ', optimum
ins, outs = cell5.get_counts()
print 'Nodes in/out: ', ins, outs

Optimum energy cell 1:  66705.0
Nodes in/out:  8875 11605
Optimum energy cell 2:  63292.0
Nodes in/out:  9913 10567
Optimum energy cell 3:  54385.0
Nodes in/out:  9674 10806
Optimum energy cell 4:  61473.0
Nodes in/out:  10340 10140
Optimum energy cell 5:  62920.0
Nodes in/out:  9663 10817


In [292]:
colors_grey = [(1.-.15*i,1.-.15*i,1.-.15*i) for i in range(6)]
colors_red = [(1.,.2,.2*i) for i in range(6)]
colors_gold = [(1.,.8,.15*i) for i in range(6)]
colors_yellow = [(1.,1.,.9-.15*i) for i in range(6)]
colors_green = [(.45,1.,.4+.1*i) for i in range(6)]
colors_blue = [(.4,1.,1.3+1*i) for i in range(6)]
colors_darkblue = [(.1,.3,1.0-.1*i) for i in range(6)]

In [293]:
#remove all meshes
while len(volwin.glWidget.meshes)>0:
    volwin.glWidget.meshes.pop(0)
volwin.glWidget.refresh()

In [294]:
volwin.glWidget.add_mesh( cell1.create_center_mesh( facecolor=colors_gold[0]) )
volwin.glWidget.add_mesh( cell2.create_center_mesh( facecolor=colors_gold[1]) )
volwin.glWidget.add_mesh( cell3.create_center_mesh( facecolor=colors_gold[2]) )
volwin.glWidget.add_mesh( cell4.create_center_mesh( facecolor=colors_gold[3]) )
volwin.glWidget.add_mesh( cell5.create_center_mesh( facecolor=colors_gold[4]) )

In [295]:
mesh1 = cell1.create_surface_mesh( facecolor=colors_gold[0] )
volwin.glWidget.add_mesh( mesh1 )

mesh2 = cell2.create_surface_mesh( facecolor=colors_gold[1] )
volwin.glWidget.add_mesh( mesh2 )

mesh3 = cell3.create_surface_mesh( facecolor=colors_gold[2] )
volwin.glWidget.add_mesh( mesh3 )

mesh4 = cell4.create_surface_mesh( facecolor=colors_gold[3] )
volwin.glWidget.add_mesh( mesh4 )

mesh5 = cell5.create_surface_mesh( facecolor=colors_gold[4] )
volwin.glWidget.add_mesh( mesh5 )

# Measure Volume

In [416]:
calibration = (1.,1.,5.)

v1 = cell1.get_volume(calibration)
v2 = cell2.get_volume(calibration)
print 'Volume of e cell 1:', v1, ' - thats equivalent to a cube of side length ', v1**(1./3)
print 'Volume of e cell 2:', v2, ' - thats equivalent to a cube of side length ', v2**(1./3)
print 

for i in range(9): #nn):
    v = cells[i].get_volume(calibration)
    print 'Volume of cell',i+1,' is ', v

#idea 2: filter all points between to results that are closer then come value d. compute area of all 
#        triangles defines by those points - touching surface!

Volume of e cell 1: 157157.5  - thats equivalent to a cube of side length  53.964940663
Volume of e cell 2: 149616.666667  - thats equivalent to a cube of side length  53.0876284648

Volume of cell 1  is  50896.6666667
Volume of cell 2  is  67134.1666667
Volume of cell 3  is  128602.5
Volume of cell 4  is  118399.166667
Volume of cell 5  is  93502.5
Volume of cell 6  is  158164.166667
Volume of cell 7  is  103726.666667
Volume of cell 8  is  101182.5
Volume of cell 9  is  48278.3333333
